###Mounting datalake gen 2 storage

In [2]:
tenantID = "1275dea2-b646-4021-8554-5a4100c6b5e0"

storageAccountName = "psdemostorageacc2"
fileSystemName = "psdemofs"

####Retrieving azure active directory app_id and app_secret from azure key vault (Enhancing security)

In [4]:

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-app-id"), # retrieving app id from the key vault
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-secret-key"), # retrieving app secret from the key vault
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/datalake",
  extra_configs = configs)


Out[3]: True

###Mounting blob storage

In [6]:
container_name = "fhvbases"
storage_account_name = "psdemoblob2"

####Retrieving azure blob storage account secret_key from azure key vault (Enhancing security)

In [8]:
dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
  mount_point = "/mnt/storage",
  extra_configs = {
      "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net" : dbutils.secrets.get(scope = "key_vault_secrets", key = "ps-demo-blob-access-key")
  }
)

Out[10]: True

###Making sure the mount is successfull by accessing the files in bith bolb storage and datalake gen 2 storage

In [10]:
dbutils.fs.ls('/mnt/storage/')

Out[1]: [FileInfo(path='dbfs:/mnt/storage/FhvBases.json', name='FhvBases.json', size=464836)]

In [11]:
dbutils.fs.head('/mnt/datalake/PaymentTypes.json')

Out[6]: '{"PaymentTypeID":1,"PaymentType":"Credit Card"}\r\n{"PaymentTypeID":2,"PaymentType":"Cash"}\r\n{"PaymentTypeID":3,"PaymentType":"No Charge"}\r\n{"PaymentTypeID":4,"PaymentType":"Dispute"}\r\n{"PaymentTypeID":5,"PaymentType":"Unknown"}\r\n{"PaymentTypeID":6,"PaymentType":"Voided Trip"}\r\n'

##The following commands are not relevant to this file. Just to know a few commands and how they work

In [13]:
%fs ls /mnt/datalake/

path,name,size
dbfs:/mnt/datalake/GreenTaxiTripData_201812.csv,GreenTaxiTripData_201812.csv,62056490
dbfs:/mnt/datalake/PaymentTypes.json,PaymentTypes.json,277
dbfs:/mnt/datalake/RateCodes.csv,RateCodes.csv,139
dbfs:/mnt/datalake/RateCodes.json,RateCodes.json,252
dbfs:/mnt/datalake/TaxiZones.csv,TaxiZones.csv,12322
dbfs:/mnt/datalake/yellow_tripdata_2018-12.csv,yellow_tripdata_2018-12.csv,721522221


In [14]:
%sh ls -l /mnt/

total 8
drwxr-xr-x 2 root root 4096 Feb 2 14:22 driver-daemon
drwxr-xr-x 2 nobody nogroup 4096 Feb 2 14:22 readonly

###Old account details and useful links (DO NOT RUN THE following cell)

In [16]:
# Mounting datalake gen 2 account using appid and service principal
'''
https://docs.databricks.com/data/data-sources/azure/azure-datalake-gen2.html#mount-an-azure-data-lake-storage-gen2-account-using-a-service-principal-and-oauth-20
'''

appID = "1e6e56a8-********"
secret = "._d57tzy*******"
tenantID = "600b1b82-******"

storageAccountName = "psdemostoacc"
fileSystemName = "psdemofs"

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": appID,
           "fs.azure.account.oauth2.client.secret": secret,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/datalake",
  extra_configs = configs)

'''
You can also access storage without mounting using

appID = "1e6e56a8-********"
secret = "._d57tzy*******"
tenantID = "600b1b82-******"

storageAccountName = "psdemostoacc"
fileSystemName = "psdemofs"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + secret + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
'''


  
  
  ----------------  ----------------  ----------------  ----------------  ----------------  ----------------  ----------------
  
  
# Mounting Blob container using access key
'''
https://docs.databricks.com/data/data-sources/azure/azure-storage.html#mount-azure-blob-storage
https://docs.databricks.com/_static/notebooks/data-sources/mount-azure-blob-storage.html
'''
container_name = "fhvbases"
storage_account_name = "psdemoblob"
storage_account_access_key = "kfnur***********"

dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
  mount_point = "/mnt/storage",
  extra_configs = {
      "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net" : storage_account_access_key
  }
)